Assignment 2:

Name:

UID:

Please submit to ELMS:
- a PDF containing all outputs (by executing **Run all**)
- your ipynb notebook containing all the code

I understand the policy on academic integrity (collaboration and the use of online materials).

Please sign your name here:


In [ ]:
## Import necessary libraries here (You can add libraries you want to use here)
import cv2
import numpy as np
from scipy.io import loadmat
from scipy import ndimage
import matplotlib.pyplot as plt
import os, sys, time
%matplotlib inline

# Part 1: A Feature Tracker (50 Points)

## Overview

In the problem, you will implement a corner detector and feature tracker that track features from the image sequence hotel. Since this is a two part problem, we have included precomputed intermediate results in the *Data* section in case you’re unable to complete any portion.

<img src="https://drive.google.com/uc?id=1sBtKpU2mYEPZ9c2Cvw-DBuLBPK2gYwC-" width="700"/>

**Note:**  Do not use existing keypoint
detectors, trackers, or structure from motion code, such as found on the web, and OpenCV.

## Data

**WARNING: Colab deletes all files everytime runtime is disconnected. Make sure to re-download the inputs when it happens.**

In [ ]:
# Download Data -- run this cell only one time per runtime
!gdown 1fT0H-FbbDZnjMfCJHZcscpcwAXHhGgNw
!unzip "/content/part1_images.zip" -d "/content/"
!gdown 1r-Pdino6MRLCEWX_sQOgd8D5AVsRc7Ym
# Load Initial Key Points
data = loadmat('/content/initial_keypoints.mat')
X0 = data['Xo']
Y0 = data['Yo']

## Helper Functions

In [ ]:
def readImages(folder, num_images):
  arr_images = []
  for i in range(num_images):
    arr_images.append(cv2.imread(f'{folder}hotel.seq{i}.png'))
  return np.array(arr_images, dtype=np.float32)

# read all 51 sequences of images
folder = '/content/part1_images/'
im = readImages(folder, 51)

## 1.1 Keypoint Selection (15 pts)

For the first frame, use the second moment matrix to locate strong corners to use as keypoints.
These points will be tracked throughout the sequence in the second part of the problem. Choose a proper threshold so that edges and noisy patches are ignored. Do local non-maxima suppression over a 5x5 window centered at each point.
This should give several hundred good points to track.

### Code (5 pts)

In [ ]:
def getKeypoints(img, tau):
  '''
  Detecting keypoints using Harris corner criterion

  Parameters:
    img: input image
    tau: threshold

  Steps:
  1. Convert to grayscale
  2. Smooth image
  3. Compute image gradients
  4. Create structure tensor M
  5. Compute corner response f
  6. Non-maximum suppression
      - Hint: might need to use ndimage.maximum_filter and .nonzero() along the way

  Output:
    keypoints : (N,2) array of [x,y] keypoints
  '''
  window_size = 7

  # Steps 1-5
  # YOUR CODE HERE

  # Step 6: Non-maximum suppression
  nms_window_size = 5
  # YOUR CODE HERE

# compute keypoints in the first sequence
tau = 0.06
key_pts = getKeypoints(im[0], tau)

# add plots for the write-up
# YOUR CODE HERE

### Write-up (10 pts)


*   (5 pts) Explain your implementation of getKeypoints()
*   (5 pts) Display the first frame of the sequence overlaid with the detected keypoints. Ensure that they are clearly visible (plot with `color='g' and linewidths=3`)



**Include your write-up here**

## 1.2 Feature Tracking (35 pts)

Apply the Kanade-Lucas-Tomasi tracking procedure to track the keypoints found in part 1.1 (or the given keypoints in the *Data* section) throughout the hotel sequence.

<img src="https://drive.google.com/uc?id=1dU4p4YcnXoQFnrNvleEty_4tDECkVW9Q" width="500"/>

Some keypoints will move out of the image frame over the course of the sequence. Discard keypoints for which the predicted translation falls outside the image frame.

**Hint:**

*  From the 1st frame to the 2nd frame, use the detected keypoints at the first frame as initialization points. From the 2nd to 3rd frame, use the tracked positions at the 2nd frame as initialization. Note that the tracked keypoints in general are not at integer positions.

*  For each point, use a window size of 15 x 15.

Add code to **plot** your tracked points overlayed in the **first sequence** and the **last sequence**. They should look similar to the second picture shown in the Overview section.



### Code (10 pts)

In [ ]:
def lucas_kanade_tracking(I1, I2, x, y, ws=5, max_iter=10, epsilon=0.01):
  '''
  Tracking using Lucas-Kanade algorithm

  Parameters:
    I1: first frame of input image sequence (grayscale)
    I2: second frame of input image sequence (grayscale)
    x: x locations of the keypoints in first frame
    y: y locations of the keypoints in first frame
    ws: local window size
    max_iter: number of iterations
    epsilon: threshold for checking convergence

  Steps:
  1. Compute image gradients
  2. Iterate until convergence
      3. Extract image gradients within local window
      4. Construct the second moment matrix A and vector b using results from (3)
      5. Solve for optical flow (u, v) using least squares
      6. Based on the (u, v) obtained in step 5, use bilinear interpolation to get sub-pixel level image gradients then solve the least square again
      7. Update keypoint locations by adding the calculated optical flow vector components
      8. Check convergence

  Output:
    new_x: newly calculated keypoints for second frame
    new_y: newly calculated keypoints for second frame
  '''
  # YOUR CODE HERE
  return new_x, new_y

# track keypoints across between 2 frames in the image sequence
new_x, new_y = lucas_kanade_tracking(image1, image2, x_initial, y_initial)


### Write-up (25 pts)

*   (5 pts) For all the keypoints, display (1) the keypoints at the first frame (as green) and (2) the tracked keypoints at the second frame (as red) on the first frame of the sequence.
*   (10 pts) For 20 random keypoints, draw the 2D path over the sequence of frames. That is, plot the progression of image coordinates for each of the 20 keypoints across all 50 images. Plot each of the paths on the same figure, overlaid on the first frame of the sequence.
*   (10 pts) On top of the first frame, plot the points which have moved out of frame at some point along the sequence. **Hint**: you may want to implement a boolean mask to keep track of which detected corners moved out of frame.





**Include your write-up here**



### Bonus (5 pts)

*   For 20 random keypoints in frame 1 and frame 10, implement coarse-to-fine registration to take care of large motion between the consecutive frames. Plot the tracked paths between frame 1 and frame 10.

In [ ]:
# YOUR CODE HERE